In [1]:
from sklearn.neighbors import LocalOutlierFactor
import numpy as np
import warnings
import copy
import os
import pandas as pd
warnings.filterwarnings("ignore")
import sys
sys.path.append('../induction_motor_anomaly_detection/')
import modules,scaler

In [2]:
normal_data = pd.read_csv('../data/combined_data.csv')
anomalous_data = pd.read_csv('../anomalous_data/anomalous_data.csv')


In [30]:
train_data_features = modules.ElectricalFeatureExtractor(current_data=normal_data)
test_data_features = modules.ElectricalFeatureExtractor(current_data=anomalous_data)
train_scaler = scaler.Scaler()
train_scaler.fit_unlabelled_data(train_data_features.feature_dataframe[0])
train_scaled = train_scaler.transform(train_data_features.feature_dataframe[0])
test_scaled = train_scaler.transform(test_data_features.feature_dataframe[0])

MinMaxScaler()


In [17]:

class AnomalyDetector:
    def __init__(self, n_neighbors=20, contamination=0.01):
        self.n_neighbors = n_neighbors
        self.contamination = contamination
        self.lof = None
        self.X = np.array([])

    def fit_predict(self, X):
        """
        Fit the LOF model to the training data.
        """
        if self.lof is None:
            self.lof = LocalOutlierFactor(n_neighbors=self.n_neighbors, contamination=self.contamination)
            self.X = X
            return self.lof.fit_predict(X)
            
        else:
            self.X = np.concatenate((self.X, X), axis=0)
            return self.lof.fit_predict(self.X)


    def fit(self, X):
        """
        Fit the LOF model to the training data.
        """
        if self.lof is None:
            self.lof = LocalOutlierFactor(n_neighbors=self.n_neighbors, contamination=self.contamination,novelty=True)
            self.lof.fit(X)
            self.X = X
        else:
            self.X = np.concatenate((self.X, X), axis=0)
            self.lof.fit(self.X)

    def predict(self, X_test):
        """
        Predict whether the data is anomalous or not based on the LOF model.
        """
        is_anomaly = self.lof.predict(X_test) == -1
        return is_anomaly


novely means that there is no training data and the model needs to learn as the data is supplied

In [ ]:
anomaly_detector = AnomalyDetector(n_neighbors=7, contamination=0.01)
anomaly_predictions = []
it = iter(train_scaled)
try:
    while True:
        if anomaly_detector.lof is None:
            data = [next(it), next(it),next(it),next(it),next(it),next(it),next(it)] #if the model is being fit for the first time
        else :
             data = [next(it)]
        anomaly_predictions =  anomaly_detector.fit_predict(data)
    
except StopIteration:
    pass


In [23]:
print(np.where(anomaly_predictions == -1)[0])


[232 235 239 242]


In [25]:
test_ano = anomaly_detector.fit_predict(test_scaled)

In [27]:
test_ano[-1]

-1